**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!apt install libgraphviz-dev pkg-config # to fix broken installation of pygraphviz
!{sys.executable} -m pip install pygraphviz==1.7
!{sys.executable} -m pip install git+https://gitlab.com/michalgregor/ani_torch.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
from ani_torch import TorchGraph, trackable_function
import matplotlib.pyplot as plt
import numpy as np
import torch

# hide a PYDEV warning triggered by the use of sys.gettrace in Google Colab
import warnings
warnings.filterwarnings('ignore', message='PYDEV DEBUGGER WARNING:.*')

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

## Automatická diferenciácia pomocou PyTorch

Ak vezmeme do úvahy, že väčšina súčasných metód strojového učenia je založená na optimalizácii a mnoho populárnych optimalizačných metódy využíva gradient (vrátane tých, čo sa využívajú v hlbokom učení), potrebujeme byť gradient schopní čo najjednoduchšie a najefektívnejšie vypočítať.

Automatická diferenciácia (autodiff; v teórii umelých neurónových sietí aj pod názvom metóda spätného šírenia chyby: backprop), je metóda, ktorá si pri výpočte gradientu zostaví graf výrazu, ktorý potom spustí dopredne (na výpočet výstupu) a spätne (na prešírenie gradientov z výstupu späť na vstup). Autodiff vie preto vypočítať gradient za cenu približne dvoch dopredných behov. Tento prístup je neporovnateľne efektívnejší než metódy, o ktorých sme hovorili doteraz: numerická diferenciácia a symbolická diferenciácia.

### Výpočtový graf a gradient

V nástroji PyTorch sa výpočtový graf konštruuje automaticky, spustením štandardného imperatívneho kódu, ale na špeciálnych objektoch. Namiesto klasických polí sa používajú PyTorch tenzory. Rovnako namiesto `numpy` operácií ako sú `np.cos` alebo `np.exp` sa požívajú PyTorch ekvivalenty `torch.cos` a `torch.exp`. Inak bude kód vyzerať prakticky rovnako.

Začneme definovaním jednoduchej PyTorch funkcie, ktorá navráti $\cos(ax + c)$:



In [ ]:
def func(x, a, c):
    y = torch.sin(a*x + c)
    return y

Aby sme funkciu mohli spustiť, potrebujeme si už len vytvoriť PyTorch tenzory. Tie sa dajú vytvoriť konverziou zo štandardných Python-ových dátových typov, prípadne aj numpy polí. Aby sme mohli určiť gradient vo vzťahu ku jednotlivým vstupom, musíme však podstúpiť dva kroky: zabezpečiť, aby mali tenzory float-ový typ a aby mali parameter `requires_grad` nastavený na `True`. Druhá podmienka vyplýva zo snahy vyhnúť sa nepotrebným výpočtom: málokedy je totiž potrebné určiť gradient vo vzťahu ku všetkým premenným.



In [ ]:
x = torch.tensor(2, dtype=float, requires_grad=True)
a = torch.tensor(3, dtype=float, requires_grad=True)
c = torch.tensor(4, dtype=float, requires_grad=True)

Ďalej funkciu spustíme na našich tenzoroch a výstup si uložíme. Môžeme ho tiež priamo vypísať.



In [ ]:
y = func(x, a, c)
print(y.item())

Na spustenie spätného behu, ktorým sa vypočítajú gradienty, stačí zavolať funkciu `y.backward()`. Gradienty sa tým prešíria na naše vstupné tenzory a vieme k nim pristupovať cez atribút `.grad`.



In [ ]:
y.backward()

print(x.grad)
print(a.grad)
print(c.grad)

### Vizualizácia výpočtového grafu

Ďalej použijeme pomocnú knižnicu na zobrazenie výpočtového grafu. Táto knižnica nie je súčasťou PyTorch: použijeme ju len na lepšiu ilustráciu toho, ako automatická diferenciácia funguje. Jediné čo treba urobiť je, že z našej funkcie a niekoľkých vstupných hodnôt vytvoríme objekt typu `TorchGraph` (vstupy môžu byť čísla alebo numpy polia – do PyTorch tenzorov sa obalia automaticky).



In [ ]:
graph = TorchGraph(func, [2, 3, 4])
graph.plot()

#### Vizualizácia dopredného a spätného behu

Čo je ešte dôležitejšie, vieme pomocou animovaného obrázka vizualizovať dopredný a spätný beh autodiff-u. To nám umožní vytvoriť vizuálne vysvetlenia toho, ako spätné šírenie gradientov funguje.



In [ ]:
graph.animate(direction="forward")

In [ ]:
graph.animate(direction="backward")

### Šírenie gradientu pre niekoľko častých prípadov

Zrejme najjednoduchším spôsobom ako porozumieť fungovaniu metódy autodiff, je prejsť si niekoľko častých prípadov ako sú ščítavanie, násobenie a pod. a vysvetliť ako sa v nich gradient šíri.

#### Sčítavanie: distribúcia gradientu

Najjednoduchším prípadom je zrejme sčítavanie: gradient z výstupu sa jednoducho distribuuje do oboch vstupných vetiev.



In [ ]:
def func_add(a, b):
    y = a + b
    return y

graph = TorchGraph(func_add, [2, 3])
graph.plot(with_all=True)

In [ ]:
graph = TorchGraph(func_add, [2, 3], [2])
graph.plot(with_all=True)

#### Súčin: výmena a násobenie

Pri súčine sa jednoducho medzi sebou vymenia vstupy z dopredného behu (a samozrejme sa násobia gradientmi z výstupu ako to vyplýva z reťazového pravidla).



In [ ]:
def func_mult(a, b):
    y = a * b
    return y

graph = TorchGraph(func_mult, [2, 3])
graph.plot(with_all=True)

In [ ]:
graph = TorchGraph(func_mult, [2, 3], [2])
graph.plot(with_all=True)

#### Vetvenia: akumulácia gradientov

Kedykoľvek sa v grafe vyskytne vetvenie a tá istá premenná sa použije viackrát, gradienty zo všetkých vetiev sa v spätnom behe akumulujú.



In [ ]:
def func_branch(x):
    y1 = torch.sqrt(x)
    y2 = torch.sqrt(x)
    return y1, y2

graph = TorchGraph(func_branch, [4], [4, 8])
graph.plot(with_all=True)

#### Operátor `max`: gradientný prepínač

Operátor `max` sa často používa ako podvzorkovacia operácia v hlbokých konvolučných sieťach. Ako sa cezeň šíria gradienty? Je zrejmé, že výstup operátora závisí len od najväčšieho vstupu. Na ten vstup sa prešíri celý gradient z výstupu. Gradienty vo vzťahu ku ostatným vstupom sú nulové: ich zmena nemá na výstup žiadny vplyv.

Dalo by sa samozrejme namietať, že hodnoty ostatných vstupov budú mať vplyv na výstup ak niektorá z nich narastie natoľko, že sa stane najväčšou. Musíme však pamätať na to, že pri výpočte gradientov nás zaujímajú nekonečne malé zmeny and nekonečne malá zmena vstupu nespôsobí, že hodnota jedného vstupu prekročí hodnotu iného.



In [ ]:
def func_branch(a, b):
    y = torch.max(a, b)
    return y

graph = TorchGraph(func_branch, [2, 5], [2])
graph.plot(with_all=True)

### Definícia nových operácií

Aby sme ešte úplnejšie pochopili, ako autodiff funguje, implementujeme si vlstnú novú operáciu: sigmoidnú funkciu. Jej matematická definícia je nasledovná:

\begin{equation}
\sigma(x) = \frac{1}{1 + e^{-x}}
\end{equation}
a jej derivácia je:

\begin{equation}
\sigma'(x) = \sigma(x) (1 - \sigma(x))
\end{equation}
Našu novú funkciu zadefinujeme ako podtriedu `torch.autograd.Function`. Definujeme v nej 
dve statické metódy (ak nerozumiete, čo to znamená, netrápte sa: len pridajte na príslušné miesto dekorátor `@staticmethod`):

* **forward:**  táto metóda realizuje dopredný beh;
* **backward:**  táto metóda realizuje spätné šírenie gradientov z výstupov našej funkcie na jej vstupy.
Je zrejmé, že výstup dopredného behu by sa dal v našom prípade opakovane použiť pri výpočte spätného behu, takže by sa nemusela opakovanie počítať tá istá výpočtovo náročná nelineárna funkcia. Výstup z dopredného behu si môžeme uložiť v kontextovom objekte `ctx` pomocou metódy `ctx.save_for_backward`.

Napokon dekorujeme aj samotnú triedu pomocou dekorátora `@trackable_function`. Tento dekorátor nie je súčasťou nástroja PyTorch: pridávame ho, aby sa naša nová funkcia dala vizualizovať. Tiež ju pomocou `"name = $\sigma$"` pomenúvame: jej názov sa bude vizualizovať ako $\sigma$ a nie ako `sigmoid`.



In [ ]:
@trackable_function
class Sigmoid(torch.autograd.Function):
    name = "$\sigma$"
    
    @staticmethod
    def forward(ctx, x):
        y = 1 / (1 + torch.exp(-x))
        ctx.save_for_backward(y)
        return y

    @staticmethod
    def backward(ctx, grad_output):
        y, = ctx.saved_tensors
        grad_input = y * (1 - y) * grad_output
        return grad_input

sigmoid = Sigmoid.apply

In [ ]:
def func_sigmoid(x):
    y = sigmoid(x)
    return y

In [ ]:
graph = TorchGraph(func_sigmoid, [2])
graph.plot()

---
### Úloha 1: Použitie autodiff-u na funkciu

**Aplikujte autodiff na nasledujúcu funkciu:** 
$$
y = a \sin(bx) + c
$$

**pričom** 
$$
a=5, b=4, c=7, x=2.
$$
**Aký je gradient voči $x$?** 

---


In [ ]:
def func(x, a, b, c):
    
    
    # ---
    
    
plt.figure(figsize=(10, 6))


# ---

